# Async Implementation

### Introduction

In the last section, we saw the benefits of working with asynchronous programming.  Essentially, we can release our Python thread from waiting and have it move onto other tasks.  We did so by implementing a coroutine in Python -- which we'll explain in more detail below.  

In this section, we'll move through the steps of implementing code with asynchronous programming.  Let's get started.

### Back to our problem

Remember that traditionally our code move sequentially, one subroutine at a time.  So for example, given the code below, first the entire `call_foursquare_api` function is completed, and then the `call_spotify_api` call is completed.

In [ ]:
import time
def call_foursquare_api():
    print("foursquare call started")
    time.sleep(2)
    print("foursquare call finished")
    
def call_spotify_api():
    print("spotify call started")
    time.sleep(1)
    print("spotify call finished")
    
call_foursquare_api()
call_spotify_api()

As we explained in the last lesson, we'll want to move to an asynchronous style, so that we can get begin making the spotify api call without waiting for the foursquare_api call to complete.  Before getting into the syntax, let's explain a bit about how asynchronous programming works.  

With *synchronous* programming, as we know our program flows sequentially.  And we can see this in the diagram below.  However, with asynchronous programming our programs don't necessarily run one after another, but instead instead our process goes in a circle checking in on each function running it when it's ready.  This is called an event loop.  

<img src="./event-loop.png" width="20%">

And you can think of the event loop like a restaurant server that checks in on each of her tables, taking the table's order when the table is ready.  Ok, now let's begin to see the implementation.

> Start reading at the very bottom, with the line `asyncio.run(main())`.

In [ ]:
import asyncio
import time

async def call_foursquare_api():
    print("foursquare call started")
    await asyncio.sleep(2)
    print("foursquare call finished")
    
async def call_spotify_api():
    print("spotify call started")
    await asyncio.sleep(1)
    print("spotify call finished")
    

async def main():
    await asyncio.gather(call_foursquare_api(), call_spotify_api())

if __name__ == "__main__":
    asyncio.run(main()) # 1. begin the event loop

Ok, so the `asyncio.run()` function is what begins the event loop.  This is what changes our python thread from running procedures one after the other, to an event loop where our thread operates like a restaurant server that checks in on various tables.

How does our event loop know which functions it needs to check in on?  Well we can see in the main function that we call two different functions with `asyncio.gather(call_foursquare_api(), call_spotify_api())`.  We can provide either asynchronous or synchronous functions to `gather` (which we'll revisit later), and this just gather call just tells the event loop to check in on these functions in that order.

<img src="./updated-loop.png" width="35%">

Ok, so let's look at the code again, this time focusing on single asynchronous function like `call_foursquare_api`.

In [ ]:
import asyncio
import time

async def call_foursquare_api():
    print("foursquare call started")
    await asyncio.sleep(2)
    print("foursquare call finished")
    
async def main():
    await asyncio.gather(call_foursquare_api())

if __name__ == "__main__":
    asyncio.run(main()) # 1. begin the event loop

There are two new keywords here.  The first is the word `async`.   This tells the event loop that the function does not need to be completed beginning to end -- but rather the function is a coroutine.  The second is the word await.  We need the await because we do want to the asynchronous procedure of sleeping to be complete before the event loop can then print the lines `foursquare_call_finished`.  And while it's waiting, the `await` allows the event loop to check in on other functions.  

So in programming speak, the thread runs the each function, and then once the `await` is reached the thread is yielded back to the event loop to see what other work can be accomplished.  The event loop pauses the execution of of the function, and schedules it to resume after the pause is complete.

### The gather function

The last item to talk about is the `asyncio.gather` function.  This function makes sure that all of the functions that are passed into gather are completed before the rest of the code is run.

So, in the example below, `done everything` won't be printed until both the `call_foursquare_api` and `call_spotify_api` functions are completed.

In [ ]:
import asyncio
import time

async def call_foursquare_api():
    print("foursquare call started")
    await asyncio.sleep(2)
    print("foursquare call finished")
    
async def call_spotify_api(): # 3. 
    print("spotify call started")
    await asyncio.sleep(1) #4. 
    print("spotify call finished") # 5. 
    

async def main():
    await asyncio.gather(call_foursquare_api(), call_spotify_api()) # 2. 
    print('done everything')

asyncio.run(main()) # 1. 

Ok, now look over the above code, and check that you can explain each of the steps.  We demarcated them for you, so that you can explain them in order.

### Summary

Ok, now let's show you our annotations of the code above.  Really try to verify that this makes sense to you.  

In [ ]:
import asyncio
import time

async def call_foursquare_api():
    print("foursquare call started")
    await asyncio.sleep(2)
    print("foursquare call finished")
    
async def call_spotify_api(): # 3. Co routine that can be paused, and resumed by event loop
    print("spotify call started")
    await asyncio.sleep(1) #4. While sleeping event loop looks for other tasks to continue with
    print("spotify call finished") # 5. Resumed when ready
    

async def main():
    await asyncio.gather(call_foursquare_api(), call_spotify_api()) # 2. Call each function in order, 
    # and only continue when all calls are complete.  Begin with foursquare call. 
    print('done everything')

asyncio.run(main()) # 1. Begin event loop, and return to async programming when complete. 